<a href="https://colab.research.google.com/github/qinkun11/colab/blob/master/deepFm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [8]:
import tensorflow as tf
print(tensorflow.__version__)

2.1.0


# 新段落

In [11]:
!pip install deepctr
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import time
from deepctr.models import DeepFM
from deepctr.inputs import SparseFeat,get_feature_names


     |████████████████████████████████| 92kB 3.4MB/s 


In [15]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
os.chdir("/content/drive/My Drive/Colab Notebooks")
time1=time.time()
data = pd.read_csv("ratings.csv")
sparse_features = ["userId", "movieId", 'timestamp']
target = ['rating']


Mounted at /content/drive


In [16]:
data["userId"]

0             1
1             1
2             1
3             1
4             1
           ... 
1048570    7120
1048571    7120
1048572    7120
1048573    7120
1048574    7120
Name: userId, Length: 1048575, dtype: int64

In [17]:

# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# 使用DeepFM进行训练
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )
# 使用DeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)
time2=time.time()
print(time2-time1)

Train on 671088 samples, validate on 167772 samples


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


671088/671088 [==============================] - 113s 168us/sample - loss: 0.9585 - mse: 0.9521 - val_loss: 0.7485 - val_mse: 0.7353
test RMSE 0.8616843969807043
201.65153670310974
